In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from glob import glob
from tqdm.notebook import tqdm
from ast import literal_eval
import os
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 22})
from sklearn.metrics import accuracy_score
from skimage import exposure
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import cv2
from matplotlib.patches import Rectangle
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, DenseNet121
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import models
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import tensorflow.keras.backend as K
from tensorflow.math import confusion_matrix
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Leemos el csv a nivel de imagen
df_image = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
display(df_image.head(5))
print(df_image.shape)

# Leemos el csv a nivel de studio con las etiquetas
df_study = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
display(df_study.head(5))
print(df_study.shape)

# Leemos el csv de sample de submission
df_sampleSub = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
display(df_sampleSub.head(5))
print(df_sampleSub.shape)

# eliminamos "_study" de los id y renombramos la columna
df_study['id'] = df_study['id'].str.replace('_study',"")
df_study.rename({'id': 'StudyInstanceUID'},axis=1, inplace=True)
df_study.head(3)

#hacemos merge de las tablas de imagen y studio por el campo StudyInstanceUID 
df_train = df_image.merge(df_study, on='StudyInstanceUID')
df_train.head(3)

# creamos columna target 'study_label' y asignamos el label correspondiente
df_train.loc[df_train['Negative for Pneumonia']==1, 'study_label'] = 'negative'
df_train.loc[df_train['Typical Appearance']==1, 'study_label'] = 'typical'
df_train.loc[df_train['Indeterminate Appearance']==1, 'study_label'] = 'indeterminate'
df_train.loc[df_train['Atypical Appearance']==1, 'study_label'] = 'atypical'

#eliminamos las variables de los casos de labels
df_train.drop(['Negative for Pneumonia','Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance'], axis=1, inplace=True)

#remplazamos en el id para dejar el formato de la imagen
df_train['id'] = df_train['id'].str.replace('_image', '.jpg')
df_train['image_label'] = df_train['label'][0].split()[0]
df_train.head(10)

In [ ]:
train_dir_jpg = '../input/covid-jpg-512/train'
train_dir_origin ='../input/siim-covid19-detection/train'
paths_original = []
paths_jpg = []
for _, row in tqdm(df_train.iterrows()):
    image_id = row['id'].split('_')[0]
    study_id = row['StudyInstanceUID']
    image_path_jpg = glob(f'{train_dir_jpg}/{image_id}.jpg')
    image_path_original = glob(f'{train_dir_origin}/{study_id}/*/{image_id}.dcm')
    paths_jpg.append(image_path_jpg)
    paths_original.append(image_path_original)
    

df_train['path'] = paths_jpg
df_train['origin'] = paths_original
df_train.head(3)

In [ ]:
df_size = pd.read_csv('../input/covid-jpg-512/size.csv')
df_size.head(3)

df_train = df_train.merge(df_size, on='id')
df_train.head(3)

In [ ]:
n = 20
train_dir = '../input/covid-jpg-512/train'
fig, axs = plt.subplots(4, 5, figsize=(20,20))
fig.subplots_adjust(hspace=.2, wspace=.2)
axs = axs.ravel()
for i in range(n):
    img = cv2.imread(os.path.join(train_dir, df_train['id'][i]))
    axs[i].imshow(img)
    if type(df_train['boxes'][i])==str:
        boxes = literal_eval(df_train['boxes'][i])
        for box in boxes:
            axs[i].add_patch(Rectangle((box['x']*(512/df_train['dim1'][i]), box['y']*(512/df_train['dim0'][i])), box['width']*(512/df_train['dim1'][i]), box['height']*(512/df_train['dim0'][i]), fill=0, color='y', linewidth=2))
            axs[i].set_title(df_train['study_label'][i])
    else:
        axs[i].set_title(df_train['study_label'][i])